In [1]:
# https://qiita.com/tizuo/items/b9af70e8cdc7fb69397f
# https://www.codexa.net/keras-lstm-cryptos-forecast/


import pandas as pd
import numpy as np
from datetime import datetime as dt
import math

from pandas.core import resample

from keras.optimizers import SGD

import numpy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.layers import LSTM

from keras.callbacks import EarlyStopping, CSVLogger

from pylab import rcParams
%matplotlib inline

#from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
input_output_columns = 5
look_back = 7

#column=[ 'Close','High','Low','HighLowDiff','OpenRelativePosition','CloseRelativePosition','ADXadx','RCIs']

column=[ 'Close','High','Low','Open','ADXadx','HighLowDiff','OpenRelativePosition','CloseRelativePosition']



In [3]:
import tensorflow as tf
tf.reset_default_graph()


In [7]:
# 為替予測

#filename = '~/Documents/ML/tensorflow/csv/GBPJPY_M15.csv'
filename = './csv/3CUR_IND/EU60.csv'

# 
df = pd.read_csv(filename,
#                nrows=100000,
                usecols=['Date','Week',
                        'Open','High','Low','Close','Volume','Profit','HighLowDiff','OpenRelativePosition','CloseRelativePosition',
                        'RCIs','RCIm','RCIl','ADXdm','ADXdp','ADXadx','MACD','MACDsig','Stocha','StochaSig'],
                dtype={ 'Date': str,
                        'Week':float,
                        'Open':float,'High':float,'Low':float,'Close':float,'Volume':float,'Profit':float,
                        'HighLowDiff':float,'OpenRelativePosition':float,'CloseRelativePosition':float,
                        'RCIs':float,'RCIm':float,'RCIl':float,'ADXdm':float,'ADXdp':float,'ADXadx':float,
                       'MACD':float,'MACDsig':float,'Stocha':float,'StochaSig':float},
                parse_dates=[0]
            ).sort_values(by='Date', ascending=True)



In [5]:
print(df.shape[0])

52496


In [8]:

# 日付と時間のデータが分かれているので、一つにつなげて日時型に変換し、インデックスに指定
#df['Datetime'] = pd.to_datetime(df['Date']+' '+df['Time'])

df.index = df['Date']
df = df[df.index.year>=2010]
df.sort_index(ascending=True)

df.dropna(inplace=True) #2つ以上のNanは削除。inplaceは自身を書き換え


In [9]:
#df['Week']=df['Week'].astype('float32')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52491 entries, 2010-01-04 00:00:00 to 2018-07-18 09:00:00
Data columns (total 21 columns):
Date                     52491 non-null datetime64[ns]
Week                     52491 non-null float64
Open                     52491 non-null float64
High                     52491 non-null float64
Low                      52491 non-null float64
Close                    52491 non-null float64
Volume                   52491 non-null float64
Profit                   52491 non-null float64
HighLowDiff              52491 non-null float64
OpenRelativePosition     52491 non-null float64
CloseRelativePosition    52491 non-null float64
RCIs                     52491 non-null float64
RCIm                     52491 non-null float64
RCIl                     52491 non-null float64
ADXdm                    52491 non-null float64
ADXdp                    52491 non-null float64
ADXadx                   52491 non-null float64
MACD                     52491 no

In [10]:
#print(df[:50])

In [11]:
#print(df.info())

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        xset = []
        for j in range(dataset.shape[1]):
            a = dataset[i:(i+look_back), j]
            xset.append(a)
        dataY.append(dataset[i + look_back, ])      
        dataX.append(xset)
    return numpy.array(dataX), numpy.array(dataY)

In [12]:
# Yは最初の列に配置
#dataframe = df.loc[:,['Close','High','Low','Volume','HighLowDiff','OpenRelativePosition','CloseRelativePosition','Week','Hourminutes']].dropna()

#dataframe = df.loc[:,[ 'Open','High','Low','Close','ADXadx','Stocha','StochaSig','HighLowDiff','OpenRelativePosition','CloseRelativePosition']].dropna()


dataframe = df.loc[:,column].dropna()


#dataframe = df.loc[:,['ExpectedPrice','Close',]].dropna()

print(dataframe[:5])   #  一番左にDateが出るがIndexであるため無視して良い

                       Close     High      Low     Open     ADXadx  \
Date                                                                 
2010-01-04 00:00:00  1.43156  1.43336  1.43151  1.43259  33.783064   
2010-01-04 01:00:00  1.42886  1.43195  1.42879  1.43156  37.266355   
2010-01-04 02:00:00  1.42705  1.42885  1.42569  1.42885  41.802726   
2010-01-04 03:00:00  1.42947  1.42989  1.42700  1.42702  43.277511   
2010-01-04 04:00:00  1.42848  1.42968  1.42718  1.42945  44.555657   

                     HighLowDiff  OpenRelativePosition  CloseRelativePosition  
Date                                                                           
2010-01-04 00:00:00      0.00185              0.167568              -0.945946  
2010-01-04 01:00:00      0.00316              0.753165              -0.955696  
2010-01-04 02:00:00      0.00316              1.000000              -0.139241  
2010-01-04 03:00:00      0.00289             -0.986159               0.709343  
2010-01-04 04:00:00      0.00

In [13]:
dataset = dataframe.values
dataset = dataset.astype('float64')
#dataset = dataset.astype('str')

from sklearn.preprocessing import MinMaxScaler

# データを標準化します。LSTMはセンシティブなので扱う数値を標準化されなければならない
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

print(scaler)

# データのうちの0.2をテスト用
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))


MinMaxScaler(copy=True, feature_range=(0, 1))
41992 10499


In [14]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [15]:
print(trainX.shape)
print(trainY.shape)

print(testX.shape)
print(testY.shape)

#print(trainX[:6]) 
print('------')
print(trainY[:6]) 

print(trainY.shape[1])

trainY_tmp=np.delete(trainY,range(input_output_columns,trainY.shape[1]),1)
testY_tmp=np.delete(testY,range(input_output_columns,testY.shape[1]),1)

print(trainX.shape)
print(trainY.shape)

trainY=trainY_tmp
testY=testY_tmp

(41984, 8, 7)
(41984, 8)
(10491, 8, 7)
(10491, 8)
------
[[ 0.86091384  0.86222067  0.86309172  0.86206897  0.42541536  0.11455847
   0.7034485   0.527586  ]
 [ 0.86281494  0.86173915  0.86063832  0.8609108   0.42073585  0.150358
   0.6921055   0.926316  ]
 [ 0.86958897  0.86865548  0.86745088  0.86281194  0.35161623  0.15234686
   0.1012985   0.9116885 ]
 [ 0.88248148  0.88104358  0.87529298  0.8698702   0.33663456  0.23508353
   0.0067455   0.9831365 ]
 [ 0.87841706  0.88200661  0.88370463  0.88252262  0.33054645  0.09984089
   0.7865615   0.0513835 ]
 [ 0.88152001  0.88568364  0.88387987  0.8783707   0.34068875  0.16348449
   0.0024215   0.355932  ]]
8
(41984, 8, 7)
(41984, 8)


In [ ]:
early_stopping = EarlyStopping(patience=40, verbose=1) 


# create and fit the LSTM network
model = Sequential()
model.add(LSTM(200, input_shape=(testX.shape[1], look_back)))
    
model.add(Dense(input_output_columns))

model.add(Activation("hard_sigmoid"))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
model_history=model.fit(trainX, trainY, epochs=500, batch_size=64, verbose=2, validation_data=(testX,testY),callbacks=[early_stopping])
#model_history=model.fit(trainX, trainY, epochs=500, batch_size=32, verbose=2, validation_data=(testX,testY))

print(model.summary())

Train on 41984 samples, validate on 10491 samples
Epoch 1/500
 - 12s - loss: 0.0055 - acc: 0.5141 - val_loss: 0.0015 - val_acc: 0.7002
Epoch 2/500
 - 11s - loss: 0.0011 - acc: 0.5922 - val_loss: 5.7661e-04 - val_acc: 0.8111
Epoch 3/500
 - 11s - loss: 5.8795e-04 - acc: 0.6765 - val_loss: 4.7433e-04 - val_acc: 0.6671
Epoch 4/500
 - 13s - loss: 4.5511e-04 - acc: 0.7236 - val_loss: 3.3193e-04 - val_acc: 0.8735
Epoch 5/500
 - 12s - loss: 4.0001e-04 - acc: 0.7498 - val_loss: 4.8498e-04 - val_acc: 0.8501
Epoch 6/500
 - 13s - loss: 3.3120e-04 - acc: 0.7610 - val_loss: 3.2577e-04 - val_acc: 0.8892
Epoch 7/500
 - 11s - loss: 2.9520e-04 - acc: 0.7509 - val_loss: 2.1989e-04 - val_acc: 0.8962
Epoch 8/500
 - 12s - loss: 2.7584e-04 - acc: 0.7586 - val_loss: 2.4846e-04 - val_acc: 0.8616
Epoch 9/500
 - 12s - loss: 2.5603e-04 - acc: 0.7551 - val_loss: 1.9395e-04 - val_acc: 0.8698
Epoch 10/500
 - 11s - loss: 2.3828e-04 - acc: 0.7617 - val_loss: 1.8499e-04 - val_acc: 0.8990
Epoch 11/500
 - 12s - loss: 2.2

In [ ]:
# モデルのsave/load
import pickle
model.save('model.h5')
with open('model.pkl', 'wb') as f:
    pickle.dump(scaler, f, protocol=pickle.HIGHEST_PROTOCOL)

    
score = model.evaluate(testX, testY, verbose=0)
print("Test score:" + str(score[0]))
print("Test accuracy:" + str(score[1]))


In [ ]:

from sklearn.metrics import mean_squared_error


# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
print(trainPredict.shape)
print(testPredict.shape)

In [ ]:


print('trainPredict.shape='+str(trainPredict.shape))
print('testPredict.shape='+str(testPredict.shape))
print('trainX.shape='+str(trainX.shape))
print('trainY.shape='+str(trainY.shape))
print('------')
print(trainX.shape[1])
print('------')


train_zero=np.resize(trainX,(trainX.shape[0],trainX.shape[1]))[:,input_output_columns:]  # scaler実行時のrow/column-1 をつくり、そこからinput_output_columnsの右の列を追加
train_zero.fill(0)
trainPredict = np.hstack([trainPredict,train_zero])
trainPredict = scaler.inverse_transform(trainPredict)

trainY = np.hstack([trainY,train_zero])
trainY = scaler.inverse_transform(trainY)


test_zero=np.resize(testX,(testY.shape[0],testX.shape[1]))[:,input_output_columns:]  # scaler実行時のrow/column-1 をつくり、そこからinput_output_columnsの右の列を追加
test_zero.fill(0)

testPredict = np.hstack([testPredict,test_zero])
testPredict = scaler.inverse_transform(testPredict)

testY = np.hstack([testY,test_zero])
testY = scaler.inverse_transform(testY)

print(trainPredict.shape)
print(testPredict.shape)


# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

acc = model_history.history['acc']
loss = model_history.history['loss']


In [ ]:
print(acc[-1])

In [ ]:
acc = model_history.history['acc']
loss = model_history.history['loss']

print(acc[-1])
print(loss[-1])

In [ ]:
print(testY[:,0])
print(testPredict[:,0])

# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(dataset)-len(testPredict):len(dataset), :] = testPredict

#testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
#testPredictPlot[len(trainPredict)+(look_back*2):len(dataset), :] = testPredict
#testPredictPlot[len(trainPredict)+(look_back*2)+2:len(dataset), :] = testPredict


plottestY=testY
plotpredict=testPredictPlot[len(testPredictPlot)-len(testY)::]

# plot baseline and predictions

rcParams['figure.figsize'] = 20,10
#plt.plot(scaler.inverse_transform(dataset))
tra=scaler.inverse_transform(dataset)
plt.plot(tra[:,0])
plt.plot(trainPredictPlot[:,0])
plt.plot(testPredictPlot[:,0])
plt.show()

In [ ]:
print(len(dataset)-len(testPredict))
print(testPredict.shape)
print(testPredict[-5:])

In [ ]:
testPredict[-5:]

In [ ]:
print(trainX.shape)
print(testX.shape)

print(trainPredict.shape)
print(testPredict.shape)

In [ ]:
fig, ax1 = plt.subplots(1,1)
 
ax1.plot(model_history.epoch, model_history.history['loss'])
ax1.set_title('TrainingError')
 
if model.loss == 'mae':
    ax1.set_ylabel('Mean Absolute Error (MAE)',fontsize=12)
# もでるおんロス計算を変更した場合のため
else:
    ax1.set_ylabel('Model Loss',fontsize=12)
ax1.set_xlabel('# Epochs',fontsize=12)
plt.show()
 

In [ ]:
# 損失の履歴をプロット
plt.clf()
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()
plt.legend(['loss', 'val_loss'], loc='lower right')
plt.show()

# チャート作成(ACC/LOSS)
plt.clf()
epochs = len(loss)
plt.plot(range(epochs), acc, marker='.', label='acc')
plt.plot(range(epochs), loss, marker='.', label='loss')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.ticker as tick # 目盛り操作に必要なライブラリを読み込みます

rcParams['figure.figsize'] = 20,60
plt.rcParams["font.size"] = 20
plt.rcParams['font.family'] ='sans-serif'


plotpredict[:,0]=(plotpredict[:,0]+plotpredict[:,1]+plotpredict[:,2])/3 #Typica Price
#plotpredict[:,0]=(plotpredict[:,1]+plotpredict[:,2])/2 #Typica Price


gs = gridspec.GridSpec(8,1)
plt.subplots_adjust(wspace=0.2, hspace=0.3)

plt.subplot(gs[0,0])
plt.grid()
plt.plot(plottestY[-400:,0],label='actual',linewidth=1.0)
plt.plot(plotpredict[-400:,0],label='predict',linewidth=1.0)
plt.title('Column:0 400 bar')
plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(10))
plt.grid(which='minor')
plt.legend()

plt.subplot(gs[1,0])
plt.grid()
plt.plot(plottestY[-100:,0],label='actual',linewidth=1.0)
plt.plot(plotpredict[-100:,0],label='predict',linewidth=1.0)
plt.title('Column:0 100 bar')
plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
plt.grid(which='minor')
plt.legend()

if len(column)>1:
    plt.subplot(gs[2,0])
    plt.grid()
    plt.plot(plottestY[-100:,1],label='actual',linewidth=1.0)
    plt.plot(plotpredict[-100:,1],label='predict',linewidth=1.0)
    plt.title('Column:1 100  bar')
    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
    plt.grid(which='minor')
    plt.legend()

if len(column)>2:
    plt.subplot(gs[3,0])
    plt.grid()
    plt.plot(plottestY[-100:,2],label='actual',linewidth=1.0)
    plt.plot(plotpredict[-100:,2],label='predict',linewidth=1.0)
    plt.title('Column:2 100 bar')
    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
    plt.grid(which='minor')
    plt.legend()

if len(column)>3:
    plt.subplot(gs[4,0])
    plt.grid()
    plt.plot(plottestY[-100:,3],label='actual',linewidth=1.0)
    plt.plot(plotpredict[-100:,3],label='predict',linewidth=1.0)
    plt.title('Column:3 100 bar')
    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
    plt.grid(which='minor')
    plt.legend()

if len(column)>4:
    plt.subplot(gs[5,0])
    plt.grid()
    plt.plot(plottestY[-100:,4],label='actual',linewidth=1.0)
    plt.plot(plotpredict[-100:,4],label='predict',linewidth=1.0)
    plt.title('Column:4 100 bar')
    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
    plt.grid(which='minor')
    plt.legend()

plt.subplot(gs[6:8:,0])
plt.grid()




plt.plot(plottestY[-60:,0],color='#b346fc',alpha=1.0,label='0 Actual',linewidth=2.0)
plt.plot(plotpredict[-60:,0],color='#b346fc',alpha=0.5,label='0 Predict',linewidth=0.5,marker='o')


if len(column)>1:
    plt.plot(plottestY[-60:,1],color='#ff033d',alpha=1.0,label='1 Actual',linewidth=2.0)
    plt.plot(plotpredict[-60:,1],color='#ff033d',alpha=0.5,label='1 Predict',linewidth=0.5,marker='o')

if len(column)>2:
    plt.plot(plottestY[-60:,2],color='#0dc4e0',alpha=1.0,label='2 Actual',linewidth=2.0)
    plt.plot(plotpredict[-60:,2],color='#0dc4e0',alpha=0.5,label='2 Predict',linewidth=0.5,marker='o')

#if len(column)>3:
#    plt.plot(plottestY[-30:,3],color='#333333',alpha=0.4,label='3 Actual',linewidth=0.5)
#    plt.plot(plotpredict[-30:,3],color='#333333',alpha=1.0,label='3 Predict',linewidth=0.5,marker='.')

#                                    plt.plot(plottestY[-30:,4],color='#56f442',alpha=0.4,label='Adx Actual',linewidth=0.5,marker='.')
#                                    plt.plot(plotpredict[-30:,4],color='#56f442',alpha=1.0,label='Adx Predict',linewidth=0.5)

plt.legend(loc='upper left')
plt.title('0-3(max) columns')
plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(1))
plt.grid(which='minor')


plt.show()


In [ ]:
df_candle = plottestY[-200:,[0,1,2]]
#print(df_candle)

In [ ]:
a=[1,2,3,4,5,6,7,8,9,10]
print(a)
print(a[2])
print(a[2::]) #offset2からすべて
print(a[:2:]) #先頭から2件
print(a[::2]) #一個飛ばし
print(a[-2:]) #最後の数件

In [ ]:
from mpl_finance import candlestick_ohlc

df_candle_testY = df.loc[:,['Close','High','Low','Open']].dropna()
df_candle_testY = df_candle_testY[-800:]

df_candle_predictY = pd.DataFrame(plotpredict[-800:,[0,1,2]])
df_candle_predictY.columns = ['Close','High','Low']
df_candle_predictY['Datetime'] = df_candle_testY.index.values
df_candle_predictY.index = df_candle_predictY['Datetime']
df_candle_predictY.drop('Datetime',axis=1,inplace=True) 

df_candle_predictY['Open']=df_candle_testY['Open'].astype('float64')
df_candle_predictY['High']=df_candle_predictY['High'].astype('float64')
df_candle_predictY['Low']=df_candle_predictY['Low'].astype('float64')
df_candle_predictY['Close']=df_candle_predictY['Close'].astype('float64')


#print(df_candle_predictY[:5])
#print(df_candle_testY.info())
#print(df_candle_predictY.info())

In [ ]:
#print(df_candle_testY[0:5:])
#print(df_candle_testY[-5:])

#print(df_candle_predictY[0:5:])
#print(df_candle_predictY[-5:])

In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly import figure_factory as FF
import plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

init_notebook_mode(connected=True) # Jupyter notebook用設定

input_candle_bars = -30

# create_candlestickのdatesを外すと predicate が表示されなくなる
fig = FF.create_candlestick(df_candle_testY.Open[input_candle_bars:], 
                            df_candle_testY.High[input_candle_bars:], 
                            df_candle_testY.Low[input_candle_bars:], 
                            df_candle_testY.Close[input_candle_bars:],
                            dates=df_candle_testY.index[input_candle_bars:],
                            direction='increasing',
                            line=Line(color='rgba(58, 249, 81,0.4)')
                           )

fig_dec = FF.create_candlestick(df_candle_testY.Open[input_candle_bars:], 
                            df_candle_testY.High[input_candle_bars:], 
                            df_candle_testY.Low[input_candle_bars:], 
                            df_candle_testY.Close[input_candle_bars:],
                            dates=df_candle_testY.index[input_candle_bars:],
                            direction='decreasing',
                            line=Line(color='rgba(232, 101, 97,0.4)')
                           )



#xtick0 = (5-df.index[0].weekday())%5 #最初の月曜日のインデックス
fig['layout'].update({
    'xaxis':{
        'showgrid': True,
#        'ticktext': [x.strftime('%Y-%m-%d %H:00') for x in df.index][xtick0::5],
#        'tickvals': np.arange(xtick0,len(df),5)
    }
})
fig['data'].extend(fig_dec['data'])
add_line = [
            go.Scatter(x=df_candle_predictY.index[input_candle_bars:], y=df_candle_predictY['High'][input_candle_bars:], name='p_high', line=Line(color = 'rgba(244, 65, 65, 1.0)',width=1)),
            go.Scatter(x=df_candle_predictY.index[input_candle_bars:], y=df_candle_predictY['Close'][input_candle_bars:], name='p_close', line=Line(color = 'rgba(90, 90, 90, 1.0)',width=1)),
            go.Scatter(x=df_candle_predictY.index[input_candle_bars:], y=df_candle_predictY['Low'][input_candle_bars:], name='p_low', line=Line(color = 'rgba(65, 76, 244, 1.0)',width=1)),
           ]
#fig['layout'].update({'xaxis':{'showgrid': True}})

fig['data'].extend(add_line)
py.offline.iplot(fig)#,filename='figure.html', image='png', auto_open=False)


#### 